In [1]:
import sys

sys.path.append("../src/lake_modelling/utils")

import matplotlib.pyplot as plt
import pandas as pd
from scipy import integrate

import lake_model as lm

plt.style.use("ggplot")

# Lake model

A notebook illustrating basic usage of the lake model.

## 1. Lake

The default is lake with an area of 0.2 km2, mean depth 5 m and residence time 0.7 years. Initial lake and inflow pH is 4.5, TOC is 4 mg/l and the flow profile is `fjell`.

Alternatively, pass `kwargs` to the constructor to define a custom lake.

**Note:** Only Altair plots are working in the notebook for now.

In [2]:
lake = lm.Lake(
    area=0.2,
    depth=5,
    tau=0.7,
    flow_prof="fjell",
    pH_lake0=4.5,
    pH_inflow=4.5,
    toc_lake0=4,
)

print(f"Volume:           {lake.volume:.0f} litres.")
print(f"Mean annual flow: {lake.mean_annual_flow:.0f} litres/month.")

lake.plot_flow_profile(lib="Altair")

Volume:           1000000000 litres.
Mean annual flow: 119047619 litres/month.


2024-05-31 10:25:45.644 
  command:

    streamlit run /opt/conda/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


alt.Chart(...)

## 2. Lime product

Either specify the name of an existing product (defined in `lime_products.xlsx`), or set `from_database=False` and pass the required arguments to the constructor to create a custom product.

In [3]:
prod = lm.LimeProduct("Miljøkalk VK3")
prod.plot_column_data(lib="Altair")

alt.HConcatChart(...)

In [4]:
# Custom product
prod = lm.LimeProduct(
    "test",
    from_database=False,
    ca_pct=40,
    mg_pct=0,
    dry_fac=0.7,
    col_depth=5,
    id_list=[70, 70, 70, 70, 70],
    od_list=[1, 1, 1, 1, 1],
)
prod.plot_column_data(lib="Altair")

alt.HConcatChart(...)

## 3. Model

Requires as input a **lake object** and a **lime product** object. 

By default, the lime dose is 10 mg/l, the liming month is 7, the spreading method is `wet`, the soluble fraction for lake-bottom lime is 1, the rate coefficient for lake-bottom lime is 0.1, the decay constant for lime becoming inactive is 0.1, the maximum concentration is 8.5 mg-Ca/l, and the number of months simulated is 12. This behaviour can be changed/overridden by passing kwargs to the constructor.

In [5]:
model = lm.Model(
    lake,
    prod,
    lime_dose=10,
    lime_month=1,
    spr_meth="wet",
    spr_prop=0.5,
    F_sol=1,
    rate_const=0.1,
    activity_const=0.1,
    ca_aq_sat=8.5,
    n_months=24,
)

print(f"Method factor for '{model.spr_meth}': {model.method_fac:.1f}.")
print(
    f"Instananeous ΔCa-equivalents: {model.C_inst0:.2f} mg/l ({model.C_inst0 * 100.09/40.08:.2f} mg-CaCO3/l)."
)
print(
    f"Slow, lake-bottom ΔCa-equivalents: {model.C_bott0:.2f} mg/l ({model.C_bott0 * 100.09/40.08:.2f} mg-CaCO3/l)."
)
model.plot_result(lib="Altair")

Method factor for 'wet': 1.0.
Instananeous ΔCa-equivalents: 1.40 mg/l (3.50 mg-CaCO3/l).
Slow, lake-bottom ΔCa-equivalents: 0.60 mg/l (1.50 mg-CaCO3/l).


alt.VConcatChart(...)

## 4. Modify and re-run

In [6]:
lake.flow_prof = "none"
model.plot_result(lib="Altair")

alt.VConcatChart(...)

In [7]:
model.lime_month = 7
model.F_sol = 1
model.lime_product = lm.LimeProduct("Microdol1")
lake.flow_prof = "fjell"
model.plot_result(lib="Altair")

alt.VConcatChart(...)

## 5. Test conservation of mass

It is important to ensure that the model satisfies the principle of **conservation of mass** i.e. that all lime entering the lake can be accounted for.

In [8]:
lake.flow_prof = "none"
model = lm.Model(
    lake,
    prod,
    lime_dose=10,
    lime_month=1,
    spr_meth="wet",
    spr_prop=1,
    F_sol=1,
    rate_const=2,  # High so that lake-bottom lime dissoles quickly
    activity_const=0,  # No decrease in activity over time
    ca_aq_sat=8.5,
    n_months=100,  # Long enough for Ca concentration to return to start
)

print(f"Method factor for '{model.spr_meth}': {model.method_fac:.1f}.")
print(
    f"Instananeous ΔCa-equivalents: {model.C_inst0:.2f} mg/l ({model.C_inst0 * 100.09/40.08:.2f} mg-CaCO3/l)."
)
print(
    f"Slow, lake-bottom ΔCa-equivalents: {model.C_bott0:.2f} mg/l ({model.C_bott0 * 100.09/40.08:.2f} mg-CaCO3/l)."
)
model.plot_result(lib="Altair")

Method factor for 'wet': 1.0.
Instananeous ΔCa-equivalents: 2.80 mg/l (6.99 mg-CaCO3/l).
Slow, lake-bottom ΔCa-equivalents: 1.20 mg/l (3.00 mg-CaCO3/l).


alt.VConcatChart(...)

In [9]:
# Total Ca-equivalents added to lake at start (in kg)
tot_in = (
    (model.lime_product.ca_pct + (model.lime_product.mg_pct * 40.08 / 24.31))
    * model.spr_prop
    * model.lake.volume
    * model.lime_dose
    / (100 * 1e6)
)

# Baseline CaCO3 level that lake returns to
base_caco3 = 0.5 # mg/l CaCO3
base_ca = base_caco3 * 40.08 / 100.09

# Total accumulated Ca from outflow by end of simulation (in kg)
# 'flow_prof' must be 'none' for this calculation to work.
dx = 0.01  # months
q = model.lake.mean_annual_flow  # l/month
load = (model.result_df["Ca (mg/l)"] - base_ca) * q * dx / 1e6  # kg
tot_out = integrate.simpson(load)
print(f"Total Ca-equivalents added:        {tot_in:.1f} kg.")
print(f"Total Ca-equivalents from outflow: {tot_out:.1f} kg.")

Total Ca-equivalents added:        4000.0 kg.
Total Ca-equivalents from outflow: 4000.0 kg.
